# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [1]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from DataLoader import generar_store_cod,generar_subgroup_cod
import xgboost as xgb
import pandas as pd

In [2]:
ventas=importar_ventas()
ventas

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,date,demand,mean_price,cluster,store_cod,subgroup_cod,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7,day,month,year
2654985,2021-01-10,1.0,319.510000,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,1,2021
4138883,2021-01-13,8.0,241.460000,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13,1,2021
3198994,2021-01-18,4.0,250.180000,0,0,0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,18,1,2021
3566508,2021-01-22,5.0,268.270000,0,0,0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,22,1,2021
965731,2021-01-23,3.0,297.770000,0,0,0,5.0,4.0,8.0,1.0,NaN,NaN,NaN,23,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4137987,2023-12-15,2.0,27.320000,8,156,73,5.0,6.0,7.0,5.0,3.0,6.0,2.0,15,12,2023
4740172,2023-12-16,15.0,29.456667,8,156,73,2.0,5.0,6.0,7.0,5.0,3.0,6.0,16,12,2023
4797741,2023-12-21,3.0,26.450000,8,156,73,15.0,2.0,5.0,6.0,7.0,5.0,3.0,21,12,2023
5194594,2023-12-26,2.0,26.530000,8,156,73,3.0,15.0,2.0,5.0,6.0,7.0,5.0,26,12,2023


In [2]:
test=preparar_test("ids_test.csv")
test

,store_subgroup_date_id,store_id,subgroup,date,store_cod,subgroup_cod,mean_price,fecha_busqueda,fecha_ref,precio_2023,day,month,year,cluster
0,S00001_Laptops_2024-01-01,S00001,Laptops,2024-01-01,0,0,0,2023-01-01,2022-12-29,318.17,1,1,2024,0
22157,S00001_Laptops_2024-01-02,S00001,Laptops,2024-01-02,0,0,0,2023-01-02,2022-12-29,318.17,2,1,2024,0
24817,S00001_Laptops_2024-01-03,S00001,Laptops,2024-01-03,0,0,0,2023-01-03,2022-12-29,318.17,3,1,2024,0
35802,S00001_Laptops_2024-01-04,S00001,Laptops,2024-01-04,0,0,0,2023-01-04,2022-12-29,318.17,4,1,2024,0
52332,S00001_Laptops_2024-01-05,S00001,Laptops,2024-01-05,0,0,0,2023-01-05,2022-12-29,318.17,5,1,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24793,S00157_Puzzles_2024-01-03,S00157,Puzzles,2024-01-03,156,73,0,2023-01-03,2023-01-01,27.67,3,1,2024,8
35761,S00157_Puzzles_2024-01-04,S00157,Puzzles,2024-01-04,156,73,0,2023-01-04,2023-01-01,27.67,4,1,2024,8
52313,S00157_Puzzles_2024-01-05,S00157,Puzzles,2024-01-05,156,73,0,2023-01-05,2023-01-01,27.67,5,1,2024,8
59669,S00157_Puzzles_2024-01-06,S00157,Puzzles,2024-01-06,156,73,0,2023-01-06,2023-01-01,27.67,6,1,2024,8


In [11]:
test.dtypes

store_subgroup_date_id     object
store_id                   object
subgroup                   object
date                       object
store_cod                   int64
subgroup_cod              float64
dtype: object

In [36]:
agregar_fourier(ventas,4)
ventas.columns


Index(['date', 'demand', 'mean_price', 'cluster', 'store_cod', 'subgroup_cod',
       'demand_lag_1', 'demand_lag_2', 'demand_lag_3', 'demand_lag_4',
       'demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 'year',
       'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_cos_1',
       'anual_sin_2', 'anual_cos_2', 'mensual_sin_2', 'mensual_cos_2',
       'anual_sin_3', 'anual_cos_3', 'mensual_sin_3', 'mensual_cos_3',
       'anual_sin_4', 'anual_cos_4', 'mensual_sin_4', 'mensual_cos_4'],
      dtype='object')

In [67]:

FEATURES=[
    'mean_price', 
    'cluster', 
     #     'store_cod', 
          'subgroup_cod',
       #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
         # 'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
          'year',
       'anual_sin_1', 
          'anual_cos_1',
          'mensual_sin_1', 
          'mensual_cos_1',
      'anual_sin_2', 
        #'anual_cos_2',
          'mensual_sin_2', 
       #   'mensual_cos_2',
          #'anual_sin_3', 
          #'anual_cos_3', 
         # 'mensual_sin_3', 
         # 'mensual_cos_3',
    #'anual_sin_4', 
    #'anual_cos_4', 
  #  'mensual_sin_4', 
 #   'mensual_cos_4'
]

TARGET=['demand']

test=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas.drop(test.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test=test[FEATURES]
y_test=test[TARGET]



In [69]:
X_test

,mean_price,cluster,store_cod,subgroup_cod,demand_lag_1,demand_lag_2,demand_lag_3,year,anual_sin_1,anual_cos_1,mensual_sin_1,mensual_cos_1,anual_sin_2,anual_cos_2,mensual_sin_2,mensual_cos_2
435,43.055000,7,0,0,5.0,1.0,2.0,2023,-0.192322,0.981332,0.616781,-0.787135,-0.377462,0.926025,-0.97098,0.239161
436,53.080000,7,0,0,6.0,5.0,1.0,2023,-0.175413,0.984495,0.616781,-0.787135,-0.345386,0.938461,-0.97098,0.239161
437,44.770000,7,0,0,4.0,6.0,5.0,2023,-0.141444,0.989946,0.616781,-0.787135,-0.280044,0.959987,-0.97098,0.239161
438,51.280000,7,0,0,5.0,4.0,6.0,2023,-0.124395,0.992233,0.616781,-0.787135,-0.246857,0.969052,-0.97098,0.239161
439,44.820000,7,0,0,3.0,5.0,4.0,2023,-0.090190,0.995925,0.616781,-0.787135,-0.179645,0.983732,-0.97098,0.239161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206788,87.750000,2,151,72,1.0,5.0,1.0,2023,-0.275913,0.961183,0.616781,-0.787135,-0.530405,0.847744,-0.97098,0.239161
5206789,72.180000,2,151,72,5.0,1.0,5.0,2023,-0.242687,0.970105,0.616781,-0.787135,-0.470863,0.882206,-0.97098,0.239161
5206790,82.050000,2,151,72,2.0,5.0,1.0,2023,-0.158452,0.987367,0.616781,-0.787135,-0.312900,0.949786,-0.97098,0.239161
5206791,84.450000,2,151,72,3.0,2.0,5.0,2023,-0.090190,0.995925,0.616781,-0.787135,-0.179645,0.983732,-0.97098,0.239161


In [68]:
from sklearn.metrics import r2_score

reg = xgb.XGBRegressor(
    n_estimators=1000,
    eval_metric=r2_score,
    eta=0.3,
    max_depth=6,
    early_stopping_rounds=600 
)

reg.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],   
    verbose=50                       
)
#sin anual_sin_2 450 estimadores =0.11283 y con da lo mismo

[0]	validation_0-rmse:3.00492	validation_0-r2_score:-0.01452
[50]	validation_0-rmse:2.88224	validation_0-r2_score:0.06663
[100]	validation_0-rmse:2.87054	validation_0-r2_score:0.07419
[150]	validation_0-rmse:2.85474	validation_0-r2_score:0.08435
[200]	validation_0-rmse:2.84214	validation_0-r2_score:0.09242
[250]	validation_0-rmse:2.84011	validation_0-r2_score:0.09372
[300]	validation_0-rmse:2.83522	validation_0-r2_score:0.09684
[350]	validation_0-rmse:2.82913	validation_0-r2_score:0.10071
[400]	validation_0-rmse:2.82843	validation_0-r2_score:0.10116
[450]	validation_0-rmse:2.82806	validation_0-r2_score:0.10139
[500]	validation_0-rmse:2.82532	validation_0-r2_score:0.10314
[550]	validation_0-rmse:2.82418	validation_0-r2_score:0.10386
[599]	validation_0-rmse:2.82424	validation_0-r2_score:0.10382


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=600,
             enable_categorical=False, eta=0.3,
             eval_metric=<function r2_score at 0x000001DB196CD080>,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None, ...)

In [64]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
year,0.159235
anual_sin_1,0.147510
cluster,0.143684
mensual_cos_1,0.133579
mensual_sin_1,0.086217
anual_cos_1,0.063806
mean_price,0.062307
mensual_sin_4,0.059133
mensual_sin_2,0.054038
subgroup_cod,0.053951


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit



X_full=pd.concat([X_train,X_test])
y_full=pd.concat([y_train,y_test])

diferenciacion=[-1]*len(X_train)+[0]*len(X_test)
ps = PredefinedSplit(diferenciacion)


reg_grid=xgb.XGBRegressor()
param_grid = {'eta': [0.2,0.3,0.4],
    'max_depth': [2,3,6,8],
     'n_estimators': [50,100,150]}



grid_search = GridSearchCV(reg_grid, 
                           param_grid, 
                           cv=ps,
                           scoring='r2')
grid_search.fit(X_full,y_full)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    i...
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'eta': [0.2, 0.3, 0.4], 'max_depth': [2, 3, 6, 8],
                         'n_estimators': [50, 100, 150]},
             scoring='r2')

In [51]:
pd.DataFrame(grid_search.cv_results_).sort_values(by="split0_test_score",ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eta,param_max_depth,param_n_estimators,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
20,59.365036,0.0,0.129849,0.0,0.3,6,150,"{'eta': 0.3, 'max_depth': 6, 'n_estimators': 150}",0.033250,0.033250,0.0,1
19,31.956481,0.0,0.091918,0.0,0.3,6,100,"{'eta': 0.3, 'max_depth': 6, 'n_estimators': 100}",0.033115,0.033115,0.0,2
18,18.879518,0.0,0.053407,0.0,0.3,6,50,"{'eta': 0.3, 'max_depth': 6, 'n_estimators': 50}",0.031518,0.031518,0.0,3
8,46.787470,0.0,0.152122,0.0,0.2,6,150,"{'eta': 0.2, 'max_depth': 6, 'n_estimators': 150}",0.029645,0.029645,0.0,4
7,32.005171,0.0,0.078285,0.0,0.2,6,100,"{'eta': 0.2, 'max_depth': 6, 'n_estimators': 100}",0.029213,0.029213,0.0,5
6,18.901373,0.0,0.039175,0.0,0.2,6,50,"{'eta': 0.2, 'max_depth': 6, 'n_estimators': 50}",0.026106,0.026106,0.0,6
17,25.284546,0.0,0.060850,0.0,0.3,3,150,"{'eta': 0.3, 'max_depth': 3, 'n_estimators': 150}",0.023289,0.023289,0.0,7
31,30.699935,0.0,0.149576,0.0,0.4,6,100,"{'eta': 0.4, 'max_depth': 6, 'n_estimators': 100}",0.017198,0.017198,0.0,8
32,50.590065,0.0,0.107938,0.0,0.4,6,150,"{'eta': 0.4, 'max_depth': 6, 'n_estimators': 150}",0.017012,0.017012,0.0,9
16,17.362342,0.0,0.037312,0.0,0.3,3,100,"{'eta': 0.3, 'max_depth': 3, 'n_estimators': 100}",0.016201,0.016201,0.0,10


In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit



X_full=pd.concat([X_train,X_test])
y_full=pd.concat([y_train,y_test])

diferenciacion=[-1]*len(X_train)+[0]*len(X_test)
ps = PredefinedSplit(diferenciacion)


reg_grid=xgb.XGBRegressor()
param_grid = {'eta': [0.3],
    'max_depth': [6],
     'n_estimators': [100]
    }



grid_search2 = GridSearchCV(reg_grid, 
                           param_grid, 
                           cv=ps,
                           scoring='r2')
grid_search2.fit(X_full,y_full)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    i...
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'eta': [0.32, 0.35, 0.37], 'max_depth': [6, 7],
                         'n_estimators': [100, 150, 200, 250]},
             scoring='r2')

In [50]:
_full.isnull().sum()

demand    0
dtype: int64

In [53]:
pd.DataFrame(grid_search2.cv_results_).sort_values(by="split0_test_score",ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eta,param_max_depth,param_n_estimators,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
11,64.972422,0.0,0.162240,0.0,0.35,6,250,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.033690,0.033690,0.0,1
19,67.307622,0.0,0.185594,0.0,0.37,6,250,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.032395,0.032395,0.0,2
8,26.112662,0.0,0.070285,0.0,0.35,6,100,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.032204,0.032204,0.0,3
10,57.557480,0.0,0.145150,0.0,0.35,6,200,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.032136,0.032136,0.0,4
9,37.514790,0.0,0.108431,0.0,0.35,6,150,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.032132,0.032132,0.0,5
17,48.622398,0.0,0.109699,0.0,0.37,6,150,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.031646,0.031646,0.0,6
16,39.857527,0.0,0.082253,0.0,0.37,6,100,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.031301,0.031301,0.0,7
18,58.700797,0.0,0.152795,0.0,0.37,6,200,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.031274,0.031274,0.0,8
3,71.232699,0.0,0.167347,0.0,0.32,6,250,"{'eta': 0.32, 'max_depth': 6, 'n_estimators': ...",0.024739,0.024739,0.0,9
1,42.043415,0.0,0.101100,0.0,0.32,6,150,"{'eta': 0.32, 'max_depth': 6, 'n_estimators': ...",0.023266,0.023266,0.0,10


In [57]:
grid_search.best_params_

{'eta': 0.3, 'max_depth': 6, 'n_estimators': 150}

In [59]:
reg=xgb.XGBRegressor(eta=0.3,max_depth=6,n_estimators=150)
reg.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.3, eval_metric=None,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=150, n_jobs=None, ...)

In [66]:
pd.DataFrame(reg.feature_importances_,index=reg.feature_names_in_,columns=["importance"]).sort_values(by="importance",
                                                                                                      ascending=False)

,importance
year,0.219976
month,0.172160
cluster,0.133384
mean_price,0.085870
demand_lag_1,0.070750
demand_lag_3,0.068313
demand_lag_2,0.060988
demand_lag_4,0.047780
demand_lag_5,0.039554
day,0.037860
